<a href="https://colab.research.google.com/github/KevinTheRainmaker/ML_DL_Basics/blob/master/HonGong_ML_DL/21_Img_Classification_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 합성곱 신경망을 이용한 이미지 분류

### **키워드:** Conv2D, MaxPooling2D, plot_model

케라스 API를 사용해 합성곱 신경망 모델을 만들어 Fashion MNIST 이미지를 분류해보고 그 모델의 층 구성을 그림으로 표현해보자.

In [1]:
# packages
import numpy as np

from tensorflow import keras
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

## Dataset

데이터를 불러오는 것은 기존의 방식과 같지만, Flatten 혹은 reshape를 사용했던 완전 연결 신경망과 달리 여기서는 2차원 이미지를 그대로 사용한다.

다만, 입력 이미지에는 깊이(채널)이 있어야 하므로 reshape를 통해 차원을 추가해주겠다. 흑백 이미지의 경우 깊이가 1이다.

In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

train_scaled = X_train.reshape(-1, 28, 28, 1) / 255.0
train_scaled, val_scaled, y_train, y_val = train_test_split(train_scaled, y_train, test_size=0.2, random_state=42)

4431872/4422102 [==============================] - 0s 0us/step


`train_scaled`의 차원이 (50000, 28, 28)에서 (50000, 28, 28, 1)이 되었다.

## 합성곱 신경망 만들기

합성곱 신경망의 구조는 기본적으로 합성곱 층으로 이미지에서 특징을 감지한 후 밀집층으로 클래스에 따른 분류 확률을 계산하는 방식을 따른다.

In [3]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu',
                              padding='same', input_shape=(28,28,1)))

32개의 필터를 사용하고 same padding을 이용하므로 위 합성곱 층의 출력은 (28, 28, 32)가 될 것이다.

`MaxPooling2D`로 풀링층을 추가해보자.

In [4]:
model.add(keras.layers.MaxPooling2D(2))

(2, 2) 최대 풀링을 적용했으므로 출력되는 특성 맵의 크기는 (14, 14, 32)가 될 것이다.

위에서 만든 첫번째 합성곱-풀링 층 다음에 두번째 합성곱-풀링 층을 추가해보자. 여기서는 더 다양하고 구체적인 특징을 감지할 수 있도록 필터의 개수를 64개로 늘리도록 하겠다.

In [5]:
model.add(keras.layers.Conv2D(64, kernel_size=3, activation='relu',
                              padding='same'))
model.add(keras.layers.MaxPooling2D(2))

합성곱 층에서는 크기가 줄지 않으나 풀링 층에서 반으로 줄어들 것이므로 최종적으로 만들어지는 특성 맵의 크기는 (7, 7, 64)가 될 것이다.

이제 최종 밀집 출력층에서 사용하기 위해서 3차원 특성 맵을 일렬고 펼쳐보겠다. 여기에서는 특성 맵을 펼쳐 바로 출력층에 전달하지 않고 하나의 밀집 은닉층을 거치도록 하겠다. 드롭아웃은 0.4로 적용하도록 하자.

In [6]:
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(10, activation='softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 100)               3